<a href="https://colab.research.google.com/github/Jacquesjh/Absenteeism/blob/main/Absenteesim_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Como sempre, começamos verificando nossa pasta de trabalho

In [15]:
%cd /content/drive/MyDrive/Machine Learning/Absenteeism

/content/drive/MyDrive/Machine Learning/Absenteeism


#Agora podemos importar as bibliotecas que usaremos

In [16]:
import pandas as pd
import numpy as np

#Podemos importar nossos dados pré processados

In [17]:
npz = np.load('Absenteeism_train_data.npz')

train_inputs = npz['inputs']
train_targets = npz['targets']

In [18]:
npz = np.load('Absenteeism_validation_data.npz')

validation_inputs = npz['inputs']
validation_targets = npz['targets']

In [19]:
npz = np.load('Absenteeism_test_data.npz')

test_inputs = npz['inputs']
test_targets = npz['targets']

In [20]:
train_inputs = np.concatenate((train_inputs, validation_inputs))

In [21]:
train_targets = np.concatenate((train_targets, validation_targets))

#Nosso modelo será um regressor logistico, iremos importá-lo do módulo de Sklearn.

In [22]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics

In [23]:
reg = LogisticRegressionCV()

In [24]:
reg.fit(train_inputs, train_targets)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

In [25]:
reg.score(train_inputs, train_targets)

0.7710651828298887

In [26]:
reg.coef_

array([[ 1.64567813e+00,  2.30337716e-01,  1.22247449e+00,
         1.00266684e+00, -1.77549508e-01,  7.09075630e-02,
         6.86585832e-01, -6.31614221e-02, -1.84242855e-01,
         3.71798566e-02,  2.81617742e-01, -6.56816054e-04,
         4.53499284e-01, -3.40517119e-01]])

##Podemos organizar estes coeficientes de uma maneira mais simples

In [27]:
coefs = pd.DataFrame(['Reason 1', 'Reason 2', 'Reason 3', 'Reason 4', 'Day of the Week', 'Month Value', 'Transportation Expense', 'Distance to Work', 'Age', 'Daily Work Load Average', 'Body Mass Index', 'Education', 'Childen', 'Pet'],)

In [28]:
coefs.columns = ['Categorias']

In [29]:
coefs['Coeficientes'] = np.transpose(reg.coef_)

In [30]:
coefs

,Categorias,Coeficientes
0,Reason 1,1.645678
1,Reason 2,0.230338
2,Reason 3,1.222474
3,Reason 4,1.002667
4,Day of the Week,-0.177550
5,Month Value,0.070908
6,Transportation Expense,0.686586
7,Distance to Work,-0.063161
8,Age,-0.184243
9,Daily Work Load Average,0.037180


In [31]:
coefs.index += 1

In [32]:
coefs.loc[0] = ['Bias', reg.intercept_[0]]

In [33]:
coefs = coefs.sort_index()
coefs

,Categorias,Coeficientes
0,Bias,-0.217284
1,Reason 1,1.645678
2,Reason 2,0.230338
3,Reason 3,1.222474
4,Reason 4,1.002667
5,Day of the Week,-0.177550
6,Month Value,0.070908
7,Transportation Expense,0.686586
8,Distance to Work,-0.063161
9,Age,-0.184243


##Por definição da classe de Sklearn, este valores de coeficientes representam o resultado em log. Podemos criar um outra coluna que represente valores fora do log

In [34]:
coefs['Coeficientes Exponenciais'] = np.exp(coefs.Coeficientes)

In [35]:
coefs

,Categorias,Coeficientes,Coeficientes Exponenciais
0,Bias,-0.217284,0.804701
1,Reason 1,1.645678,5.184525
2,Reason 2,0.230338,1.259025
3,Reason 3,1.222474,3.395580
4,Reason 4,1.002667,2.725541
5,Day of the Week,-0.177550,0.837320
6,Month Value,0.070908,1.073482
7,Transportation Expense,0.686586,1.986920
8,Distance to Work,-0.063161,0.938792
9,Age,-0.184243,0.831734


In [36]:
##Agora podemos representar a importância das categorias por relevância
coefs = coefs.sort_values('Coeficientes Exponenciais', ascending = False)

##As categorias que mais impactam nosso problema possuem coeficientes o mais diferente de 0. Assim, podemos excluir categorias como 'Education', 'Distance to Work', 'Month Value' e 'Daily Work Load Average'.

In [37]:
coefs.drop([6, 10, 12, 8])

,Categorias,Coeficientes,Coeficientes Exponenciais
1,Reason 1,1.645678,5.184525
3,Reason 3,1.222474,3.395580
4,Reason 4,1.002667,2.725541
7,Transportation Expense,0.686586,1.986920
13,Childen,0.453499,1.573810
11,Body Mass Index,0.281618,1.325272
2,Reason 2,0.230338,1.259025
5,Day of the Week,-0.177550,0.837320
9,Age,-0.184243,0.831734
0,Bias,-0.217284,0.804701


#Podemos finalmente testar o nosso modelo no nosso test_data

In [38]:
reg.score(test_inputs, test_targets)

0.8142857142857143

In [39]:
predict_prob = reg.predict_proba(test_inputs)

In [40]:
predict_prob.shape

(70, 2)

In [41]:
predict_prob[:, 1]

array([0.29949069, 0.26268217, 0.79601296, 0.62142465, 0.02725519,
       0.28619184, 0.32525952, 0.15248923, 0.6258141 , 0.66409705,
       0.4737961 , 0.91431342, 0.25892413, 0.44085235, 0.7871381 ,
       0.17209664, 0.39265449, 0.22083749, 0.28734412, 0.6267798 ,
       0.64748258, 0.47849782, 0.20810421, 0.17130168, 0.88118886,
       0.3259406 , 0.65275891, 0.5512369 , 0.74188642, 0.26361825,
       0.18512828, 0.89829931, 0.73286843, 0.25912684, 0.49560696,
       0.49686776, 0.36275631, 0.5962863 , 0.72902536, 0.61608948,
       0.62764199, 0.84040986, 0.73565547, 0.4646263 , 0.75946308,
       0.11538735, 0.79275895, 0.77750695, 0.61436368, 0.91290527,
       0.21924231, 0.73380994, 0.83428633, 0.14470241, 0.37144801,
       0.5224835 , 0.48031996, 0.42391447, 0.64918763, 0.52163378,
       0.15174261, 0.3108977 , 0.17891217, 0.51785723, 0.14196324,
       0.15055359, 0.3754276 , 0.67522255, 0.71597394, 0.45952789])

#Com isso, podemos criar um modelo de regressão logística que consegue determinar com 81% de precisão se um indivíduo ficara excessivamente ausente do trabalho

#Podemos agora salvar nosso modelo para futuro uso

In [42]:
##A biblioteca pickle salva objetos em Python de forma muito leve
import pickle

In [43]:
with open('Logistic Regression Model', 'wb') as file:
  pickle.dump(reg, file)

###DETALHE: Nós JÁ salvamos nosso escalador na parte de pré processamento